In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#  In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?
# Casper

 

In [2]:
df_movies = pd.read_csv('../../data/movie.csv')
df_movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## 2. Data processing 
Alleen 'keywords' en 'genres' nodig.

In [3]:

# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Movie FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(11, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')


## 3. Data Cleaning

In [4]:

# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard

# Negatieve waardes verwijderen
num = df_movies._get_numeric_data()
num[num < 0] = 0


## Processing: Eigen toevoeging: TODO

In [5]:
df_Q2 = df_movies.loc[:, ("Genres", "Plot Keywords")]
df_Q2.head()

,Genres,Plot Keywords
4810,Drama|History|War,huguenot|intolerance|medicis|protestant|wedding
4958,Crime|Drama,family relationships|gang|idler|poorhouse|thief
4885,Drama|Romance|War,chewing gum|climbing a tree|france|translation...
2734,Drama|Sci-Fi,art deco|bible quote|dance|silent film|worker
4812,Musical|Romance,sibling rivalry|singer|sister act|whistling|wi...


## Cleaning: Eigen toevoeging: TODO

In [6]:
# All '0' are missing values, remove
df_Q2.replace("0", np.NaN, inplace=True)
df_Q2.dropna(inplace=True)

In [7]:
df_Q2.loc[:, "Split Genres"] = df_Q2["Genres"].str.split(pat="|")
df_Q2.loc[:, "Split Keywords"] = df_Q2["Plot Keywords"].str.split(pat="|")

df_Q2.head()

,Genres,Plot Keywords,Split Genres,Split Keywords
4810,Drama|History|War,huguenot|intolerance|medicis|protestant|wedding,"[Drama, History, War]","[huguenot, intolerance, medicis, protestant, w..."
4958,Crime|Drama,family relationships|gang|idler|poorhouse|thief,"[Crime, Drama]","[family relationships, gang, idler, poorhouse,..."
4885,Drama|Romance|War,chewing gum|climbing a tree|france|translation...,"[Drama, Romance, War]","[chewing gum, climbing a tree, france, transla..."
2734,Drama|Sci-Fi,art deco|bible quote|dance|silent film|worker,"[Drama, Sci-Fi]","[art deco, bible quote, dance, silent film, wo..."
4812,Musical|Romance,sibling rivalry|singer|sister act|whistling|wi...,"[Musical, Romance]","[sibling rivalry, singer, sister act, whistlin..."


In [8]:
df_Q2.drop(["Genres", "Plot Keywords"], axis=1, inplace=True)

In [9]:
df_Q2.head()

,Split Genres,Split Keywords
4810,"[Drama, History, War]","[huguenot, intolerance, medicis, protestant, w..."
4958,"[Crime, Drama]","[family relationships, gang, idler, poorhouse,..."
4885,"[Drama, Romance, War]","[chewing gum, climbing a tree, france, transla..."
2734,"[Drama, Sci-Fi]","[art deco, bible quote, dance, silent film, wo..."
4812,"[Musical, Romance]","[sibling rivalry, singer, sister act, whistlin..."


## 4. Data Exploration & Analysis

In [10]:
pd.DataFrame([df_Q2["Split Genres"].str.len().describe(), df_Q2["Split Keywords"].str.len().describe()])

,count,mean,std,min,25%,50%,75%,max
Split Genres,4811.0,2.871337,1.200256,1.0,2.0,3.0,4.0,8.0
Split Keywords,4673.0,4.930452,0.460884,1.0,5.0,5.0,5.0,5.0


Hoeveel genres / Keywords gebruiken we?

In [11]:
def find_uniques(se, dtype = "category"):
    name = se.name
    flatten = lambda l: [item for sublist in l for item in sublist]
    
    unique_values = se.values
    unique_values = np.unique(flatten(unique_values), return_counts=True)
    
    d = {name : unique_values[0], "Count" : unique_values[1]}
    types = {name : dtype, "Count" : "int32"}
    
    df_unique = pd.DataFrame(d).astype(types)
    
    return df_unique.sort_values("Count", ascending=False)

find_uniques(df_Q2["Split Genres"])

,Split Genres,Count
7,Drama,2459
4,Comedy,1810
21,Thriller,1336
0,Action,1094
17,Romance,1067
1,Adventure,878
5,Crime,837
18,Sci-Fi,577
9,Fantasy,569
12,Horror,532
